# Quantopian Enterprise - Sample Aqueduct Usage
***
Aqueduct is an HTTP API that allows you to run and retrieve pipelines outside of the Quantopian platform.

Models developed in Quantopian Enterprise can be run via HTTP requests to the Aqueduct server without any modification, streamlining the path from research to production.

For more information on Aqueduct, including installation, please visit the [Setup](https://factset.quantopian.com/docs/user-guide/tools/aqueduct#setup) in Quantopian Enterprise.

In [1]:
import time
import pandas as pd

import aqueduct_client
from aqueduct_client import create_client
from aqueduct_client.utils import load_api_key

## Running the AqueductClient

Once the `aqueduct_client` and api key have been loaded, we can create an instance of the client.

In [2]:
API_KEY = load_api_key()
client = create_client(API_KEY)
print("Aqueduct client version: {}".format(aqueduct_client.__version__))

Aqueduct client version: 0.3


Import a Pipeline query and invoke the `submit_pipeline_execution` method

In [3]:
# Open pipeline_query.py, a Python file 
# in the same directory as this notebook 
# that contains our pipeline definition.
file = open("pipeline_query.py", "r")
code = file.read()

pipeline_id = client.submit_pipeline_execution(
    code, 
    start_date="2015-12-31", 
    end_date="2017-12-31", 
    name="MyPipeline"
)

pipeline_id

'5e165d2ca809d1660ad1910e'

Run Pipeline Query

In [4]:
while True:
    status = client.get_pipeline_execution(pipeline_id)["status"]
    print(status)
    if status == "SUCCESS":
        break
    if status == "FAILED":
        print(client.get_pipeline_execution_error(pipeline_id))
        break
    time.sleep(10)

SUCCESS


Use the `get_pipeline_results_dataframe` method to pull the results.

In [5]:
result = client.get_pipeline_results_dataframe(pipeline_id)

Preview the results

In [6]:
result.head()

ann_vol        ey      fcfy       gpa  log_mcap    lt_mom
date       sid                                                            
2015-12-31 2    1.401180  2.237009  0.578636 -0.826188 -0.685427 -1.867208
           24  -0.303392  0.143422  1.312154  0.256917  2.293536  0.158758
           62  -0.391409 -0.700045 -0.144533  0.035006  1.230461 -0.050641
           67   0.447439 -1.580643 -0.173034  0.743537 -0.707037  0.145239
           76  -0.130972 -1.241548 -0.363105 -0.762339 -0.446453  0.997855

In addition to submitting new pipeline runs to Aqueduct, the client also gives you access to the results of all Aqueduct runs you have submitted in the past. To get a list of all past results, use the `get_all_pipeline_executions` method.

In [7]:
all_pipelines = client.get_all_pipeline_executions()

historical_pipelines = pd.DataFrame(all_pipelines).sort_values(
    "created_at", ascending=False
)
historical_pipelines = historical_pipelines[
    [
        "created_at",
        "id",
        "name",
        "asset_identifier_format",
        "start_date",
        "end_date",
        "status",
        "output_format",
    ]
]

historical_pipelines.head(1)

,created_at,id,name,asset_identifier_format,start_date,end_date,status,output_format
761,2020-01-08T22:52:28.675448,5e165d2ca809d1660ad1910e,MyPipeline,sid,2015-12-31,2017-12-31,SUCCESS,csv


Historical, point-in-time Aqueduct results can be retrieved by selecting the **IDs** shown above. Below is an example:

In [8]:
historical_pipeline_id = "5e165d2ca809d1660ad1910e"

while True:
    status = client.get_pipeline_execution(historical_pipeline_id)["status"]
    print(status)
    if status == "SUCCESS":
        historical_result = client.get_pipeline_results_dataframe(
            historical_pipeline_id
        )
        break
    if status == "FAILED":
        print(client.get_pipeline_execution_error(historical_pipeline_id))
        break
    time.sleep(10)

SUCCESS


Preivew the results

In [9]:
historical_result.head()

ann_vol        ey      fcfy       gpa  log_mcap    lt_mom
date       sid                                                            
2015-12-31 2    1.401180  2.237009  0.578636 -0.826188 -0.685427 -1.867208
           24  -0.303392  0.143422  1.312154  0.256917  2.293536  0.158758
           62  -0.391409 -0.700045 -0.144533  0.035006  1.230461 -0.050641
           67   0.447439 -1.580643 -0.173034  0.743537 -0.707037  0.145239
           76  -0.130972 -1.241548 -0.363105 -0.762339 -0.446453  0.997855